In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import pymongo
from pymongo import MongoClient
from ast import literal_eval
from datetime import datetime

In [2]:
### get data from mongodb
client = MongoClient()

#point the client at mongo URI
client = MongoClient('localhost:27017')
#select database
db = client['Big-Data-DB']
#select the collection within the database
tweets = db.Tweets
tweets = pd.DataFrame(tweets.find())

In [3]:
# preprocessing
# split probabilities into their own columns
tweets['positive_value'] = tweets['sentiment_probabilities'].apply(lambda x: np.array(literal_eval(x))[0][0][1] if x.startswith('[[[\'positive\',') else np.NaN)
tweets['negative_value'] = tweets['sentiment_probabilities'].apply(lambda x: np.array(literal_eval(x))[0][1][1] if x.startswith('[[[\'positive\',') else np.NaN)
tweets['neutral_value'] = tweets['sentiment_probabilities'].apply(lambda x: np.array(literal_eval(x))[0][2][1] if x.startswith('[[[\'positive\',') else np.NaN)

# turn timestamp string to date
tweets['timestamp'] = tweets['timestamp'].apply(lambda x: datetime.fromtimestamp(float(x)) if (len(x) == 17 and x[10] == '.' and x[0:9].isnumeric() and x[11:16].isnumeric()) else np.NaN)
# turn sentiment to categorical value
tweets['sentiment'] = tweets['sentiment'].apply(lambda x: x[2:len(x)-2] if (x == '[\'neutral\']' or x == '[\'negative\']' or x == '[\'positive\']') else np.NaN)
tweets['sentiment'] = tweets['sentiment'].astype('category')

In [7]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3367 entries, 0 to 3366
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   _id                      3367 non-null   object        
 1   tweet                    3367 non-null   object        
 2   timestamp                2975 non-null   datetime64[ns]
 3   sentiment                3309 non-null   category      
 4   sentiment_probabilities  3367 non-null   object        
 5   positive_value           3309 non-null   object        
 6   neutral_value            3309 non-null   object        
 7   negative_value           3309 non-null   object        
dtypes: category(1), datetime64[ns](1), object(6)
memory usage: 187.7+ KB


In [4]:
profile = ProfileReport(tweets, title="Tweet Report")

In [5]:
#profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Value '280.2162162162162' should be a ratio between 1 and 0.

In [8]:
profile.to_file("tweet_report.html")

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: Value '280.2162162162162' should be a ratio between 1 and 0.

In [3]:
from germansentiment import SentimentModel
model = SentimentModel()
classes, probabilities = model.predict_sentiment(['lol'], output_probabilities=True)
print(probabilities[0][0])

['positive', 0.9642674326896667]


In [8]:
probs = ''
for i in range(3):
     probs += str(probabilities[0][i][1]) + ';'
print(probs)

0.9642674326896667;0.03554828092455864;0.00018431071657687426;
